In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-ibm-transpiler

# Pases de transpilador con IA
Los pases de transpilador impulsados por IA son pases que funcionan como un reemplazo directo de los pases "tradicionales" de Qiskit para algunas tareas de transpilación. Suelen producir mejores resultados que los algoritmos heurísticos existentes (como una profundidad y cantidad de CNOT menores), pero también son mucho más rápidos que los algoritmos de optimización como los solvers de satisfactibilidad booleana. Los pases de transpilador con IA pueden ejecutarse en tu entorno local o en la nube usando el Qiskit Transpiler Service si eres parte del Plan Premium, Plan Flex o Plan On-Prem (a través de la API de IBM Quantum Platform) de IBM Quantum&reg;.

> **Note:** Los pases de transpilador impulsados por IA están en estado de versión beta, sujetos a cambios.
>     Si tienes comentarios o quieres contactar al equipo de desarrollo, usa este [canal del Slack de Qiskit](https://qiskit.slack.com/archives/C06KF8YHUAU).

Los siguientes pases están disponibles actualmente:

**Pases de enrutamiento (Routing)**
 - `AIRouting`: Selección del diseño (layout) y enrutamiento del circuito

**Pases de síntesis de circuitos**
 - `AICliffordSynthesis`: Síntesis de circuitos Clifford
 - `AILinearFunctionSynthesis`: Síntesis de circuitos de funciones lineales
 - `AIPermutationSynthesis`: Síntesis de circuitos de permutación
 - `AIPauliNetworkSynthesis`: Síntesis de circuitos de Redes de Pauli (Pauli Network)

Para usar los pases de transpilador con IA, primero [instala el paquete `qiskit-ibm-transpiler`](/guides/qiskit-transpiler-service#install-transpiler-service). Visita la [documentación de la API de qiskit-ibm-transpiler](https://docs.quantum.ibm.com/api/qiskit-ibm-transpiler) para obtener más información sobre las diferentes opciones disponibles.

## Ejecutar los pases de transpilador con IA localmente o en la nube
Si quieres usar los pases de transpilador impulsados por IA en tu entorno local de forma gratuita, instala `qiskit-ibm-transpiler` con algunas dependencias extra de la siguiente manera:

In [1]:
from qiskit.transpiler import PassManager
from qiskit.circuit.library import efficient_su2
from qiskit_ibm_transpiler.ai.routing import AIRouting
from qiskit_ibm_runtime import QiskitRuntimeService

backend = QiskitRuntimeService().backend("ibm_torino")
ai_passmanager = PassManager(
    [
        AIRouting(
            backend=backend,
            optimization_level=2,
            layout_mode="optimize",
            local_mode=True,
        )
    ]
)


circuit = efficient_su2(101, entanglement="circular", reps=1)

transpiled_circuit = ai_passmanager.run(circuit)

Sin estas dependencias adicionales, los pases de transpilador con IA se ejecutan en la nube a través del Qiskit Transpiler Service (disponible solo para usuarios del Plan Premium, Plan Flex o Plan On-Prem (a través de la API de IBM Quantum Platform) de IBM Quantum). Después de instalar las dependencias extra, el modo predeterminado para ejecutar los pases de transpilador con IA es usar tu máquina local.

## Pase de enrutamiento con IA
El pase `AIRouting` actúa tanto como una etapa de diseño (layout) como una etapa de enrutamiento. Puede ser usado dentro de un `PassManager` de la siguiente manera:

In [ ]:
from qiskit.transpiler import PassManager

from qiskit_ibm_transpiler.ai.routing import AIRouting
from qiskit_ibm_transpiler.ai.synthesis import AILinearFunctionSynthesis
from qiskit_ibm_transpiler.ai.collection import CollectLinearFunctions
from qiskit_ibm_transpiler.ai.synthesis import AIPauliNetworkSynthesis
from qiskit_ibm_transpiler.ai.collection import CollectPauliNetworks
from qiskit.circuit.library import efficient_su2

ibm_torino = QiskitRuntimeService().backend("ibm_torino")
ai_passmanager = PassManager(
    [
        AIRouting(
            backend=ibm_torino,
            optimization_level=3,
            layout_mode="optimize",
            local_mode=True,
        ),  # Route circuit
        CollectLinearFunctions(),  # Collect Linear Function blocks
        AILinearFunctionSynthesis(
            backend=ibm_torino, local_mode=True
        ),  # Re-synthesize Linear Function blocks
        CollectPauliNetworks(),  # Collect Pauli Networks blocks
        AIPauliNetworkSynthesis(
            backend=ibm_torino, local_mode=True
        ),  # Re-synthesize Pauli Network blocks.
    ]
)

circuit = efficient_su2(10, entanglement="full", reps=1)

transpiled_circuit = ai_passmanager.run(circuit)

Aquí, el `backend` determina para qué mapa de acoplamiento enrutar, el `optimization_level` (nivel de optimización 1, 2 o 3) determina el esfuerzo computacional a gastar en el proceso (mayor usualmente da mejores resultados pero tarda más), y el `layout_mode` (modo de diseño) especifica cómo manejar la selección del layout.
El `layout_mode` incluye las siguientes opciones:

- `keep`: Respeta la distribución (layout) establecida por pases del transpilador anteriores (o usa la distribución trivial si no se ha establecido). Típicamente solo se usa cuando el circuito debe ejecutarse en qubits específicos del dispositivo. A menudo produce resultados peores porque tiene menos margen para optimización.
- `improve`: Esto usa el layout configurado por los pases del transpilador anteriores como un punto de partida. Es muy útil cuando tienes una buena estimación inicial para la distribución; por ejemplo, para circuitos que se construyen de una forma que sigue aproximadamente el mapa de acoplamiento del dispositivo. También es útil si quieres probar otros pases de layout específicos combinados con el pase `AIRouting`.
- `optimize`: Este es el modo predeterminado. Funciona mejor para circuitos generales donde quizás no tengas buenas aproximaciones de layout. Este modo ignora las selecciones de layout anteriores.
- `local_mode`: Esta bandera determina dónde se ejecuta el pase `AIRouting`. Si es `False`, `AIRouting` se ejecuta remotamente a través del Qiskit Transpiler Service. Si es `True`, el paquete intenta ejecutar el pase en tu entorno local con una alternativa al modo nube si no se encuentran las dependencias requeridas.

## Pases de síntesis de circuitos con IA
Los pases de síntesis de circuitos impulsados por IA te permiten optimizar partes de diferentes tipos de circuitos ([Clifford](https://docs.quantum.ibm.com/api/qiskit/qiskit.quantum_info.Clifford), [Funciones Lineales](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.LinearFunction), [Permutación](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.Permutation#permutation), Redes de Pauli) resintetizándolos. Una forma típica de usar el pase de síntesis es la siguiente:

In [ ]:
from qiskit_ibm_transpiler import generate_ai_pass_manager
from qiskit.circuit.library import efficient_su2
from qiskit_ibm_runtime import QiskitRuntimeService


backend = QiskitRuntimeService().backend("ibm_torino")
torino_coupling_map = backend.coupling_map


su2_circuit = efficient_su2(101, entanglement="circular", reps=1)

ai_transpiler_pass_manager = generate_ai_pass_manager(
    coupling_map=torino_coupling_map,
    ai_optimization_level=3,
    optimization_level=3,
    ai_layout_mode="optimize",
)

ai_su2_transpiled_circuit = ai_transpiler_pass_manager.run(su2_circuit)

La síntesis respeta el mapa de acoplamiento de los dispositivos: puede ejecutarse de forma segura después de otros pases de enrutamiento sin alterar el circuito, por lo tanto el circuito completo continuará siguiendo las restricciones del dispositivo. Por defecto, la síntesis reemplazará el sub-circuito original solo si el sub-circuito sintetizado mejora al original (actualmente solo verifica el recuento de compuertas CNOT), sin embargo, esto se puede forzar para reemplazar siempre el circuito estableciendo `replace_only_if_better=False`.

Los siguientes pases de síntesis están disponibles desde `qiskit_ibm_transpiler.ai.synthesis`:

- *AICliffordSynthesis*: Síntesis para circuitos [Clifford](https://docs.quantum.ibm.com/api/qiskit/qiskit.quantum_info.Clifford) (bloques de compuertas `H`, `S` y `CX`). Actualmente soporta bloques de hasta nueve qubits.
- *AILinearFunctionSynthesis*: Síntesis para circuitos de [Función Lineal (Linear Function)](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.LinearFunction) (bloques de puertas `CX` y `SWAP`). Actualmente soporta bloques de hasta nueve qubits.
- *AIPermutationSynthesis*: Síntesis para circuitos de [Permutación](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.Permutation#permutation) (bloques de puertas `SWAP`). Actualmente está disponible para bloques de 65, 33 y 27 qubits.
- *AIPauliNetworkSynthesis*: Síntesis para circuitos de Red de Pauli (Pauli Network) (bloques de puertas `H`, `S`, `SX`, `CX`, `RX`, `RY` y `RZ`). Actualmente soporta bloques de hasta seis qubits.

Esperamos aumentar gradualmente el tamaño de los bloques compatibles.

Todos los pases usan un pool de hilos para mandar diversas solicitudes en paralelo. De forma predeterminada, la cantidad máxima de hilos es la cantidad de núcleos más cuatro (valores predeterminados de la clase `ThreadPoolExecutor` en Python). A pesar de esto, tú puedes configurar tu propio valor usando el argumento `max_threads` cuando instancias el pase. Por ejemplo, el siguiente segmento muestra cómo crear una instancia del pase `AILinearFunctionSynthesis` para que use hasta un máximo de 20 hilos.